In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly
plotly.tools.set_credentials_file(username='ninjamouse', api_key='LejqbK7NWhPU7zXoMuhw')

import numpy as np
import pandas as pd

df = pd.read_csv('first_attack.csv')

In [2]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,"2010-06-12 06:50:00,055616",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
1,2,"2010-06-12 06:50:00,059867",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
2,3,"2010-06-12 06:50:00,061151",192.168.4.120,88.208.248.162,TCP,60,2677 > 80 [ACK] Seq=1 Ack=2921 Win=17520 Len=0
3,4,"2010-06-12 06:50:00,062786",208.122.28.2,192.168.4.120,HTTP,1195,HTTP/1.1 200 OK (JPEG JFIF image)
4,5,"2010-06-12 06:50:00,064234",192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...


In [3]:
#df.sort_values(['Source', 'Destination'],  ascending=[True, False]  )
#df['Info'] = df['Info'].astype('str')
#get=df[df['Info'].str.contains('GET')]


#Sort the connections based on the pair Source-->Destination
#Select all the connections that have Source IP and Destination IP with 75.127.97.72
df.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()
d=df[(df.Destination=='75.127.97.72') | (df.Source=='75.127.97.72')]   

In [4]:
d['Info'] = d['Info'].astype('str')
d[d['Info'].str.contains('GET')].head()

,No.,Time,Source,Destination,Protocol,Length,Info
31176,31177,"2010-06-12 06:54:06,348044",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/ HTTP/1.1
31192,31193,"2010-06-12 06:54:06,585868",192.168.56.1,75.127.97.72,HTTP,749,GET /drupal7/sites/default/files/styles/medium...
31203,31204,"2010-06-12 06:54:06,586953",192.168.56.1,75.127.97.72,HTTP,734,GET /drupal7/sites/default/files/styles/medium...
31206,31207,"2010-06-12 06:54:06,587263",192.168.56.1,75.127.97.72,HTTP,733,GET /drupal7/sites/default/files/styles/medium...
31208,31209,"2010-06-12 06:54:06,587412",192.168.56.1,75.127.97.72,HTTP,684,GET /drupal7/misc/feed.png HTTP/1.1


In [5]:
trace1=go.Scatter(x=d['Time'],y=d['Length'],mode='markers')
l=d.query('(Length>300)&(Length<600)')
trace2=go.Scatter(x=l['Time'],y=l['Length'], mode='lines')
data=[trace1, trace2]
py.iplot(data)

/home/deepl/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [6]:
from sklearn import linear_model

In [7]:
X = df

# Treat the problem as a pure classification problem.

X['Protocol'] = X['Protocol'].astype('category')
# Could use float, but it doesn't make much sense
#X['Source'] = X['Source'].apply(lambda x: float(x.replace('.','')))

# Convert IPs to classes
X['Destination'] = X['Destination'].astype('category')
X['Source'] = X['Source'].astype('category')

# Convert info to classes as well
X['Info'] = X['Info'].astype('category')  # ToDo split info in relevant columns

# Drop the time series for now, just to play with a classifier
X = X.drop(['Time'], axis=1)
X.head()

,No.,Source,Destination,Protocol,Length,Info
0,1,88.208.248.162,192.168.4.120,HTTP,1514,Continuation
1,2,88.208.248.162,192.168.4.120,HTTP,1514,Continuation
2,3,192.168.4.120,88.208.248.162,TCP,60,2677 > 80 [ACK] Seq=1 Ack=2921 Win=17520 Len=0
3,4,208.122.28.2,192.168.4.120,HTTP,1195,HTTP/1.1 200 OK (JPEG JFIF image)
4,5,192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...


In [8]:
# Set some entries as 1 'attack' and 0 no attack
# The ones you defined in d above, are entries where an attack was plausible. Everything else is clean
Y = [0 if i not in l.index.values else 1 for i in df.index.values]

In [9]:
#1. Aici ce facem? Pentru fiecare valoare este asignata nu cod?
## Transformam toate coloanele care nu sunt numerice, in valori numerice.
## De exemplu, pentru coloana Protocol ai valori 'HTTP' si 'TCP'. Matematic e greu sa le analizezi
## daca nu sunt in format numeric. Asa ca functia lambda de mai jos iti transforma
## toate valorile 'HTTP' in 0 si 'TCP' in 1 (de exemplu). Daca ai mai multe coloane, ai mai multe valori (0,1,2,3..)
catColumns = X.select_dtypes(['category']).columns
X[catColumns] = X[catColumns].apply(lambda x: x.cat.codes)

In [10]:
X.columns

Index(['No.', 'Source', 'Destination', 'Protocol', 'Length', 'Info'], dtype='object')

In [11]:
#2. Banuiesc ca aici este aplicat algoritmul de linear regression? Ar trebui sa il inteleg sau doar il aplic si gata?
## Aici faci doar partea de model fitting a algoritmului. Pe baza valorilor din X, iti antrenezi modelul ca sa prezica
## valorile din Y. Aici calculezi coeficientii cei mai potriviti din formula liniara care iti satisfac cel mai bine
## X = Y
## sau mai in detaliu, ai avea 
## coef_Source * X['Source'] + coef_Destination * X['Destination'] + .. + coef_Info * X['Info'] = Y
## ,pentru fiecare linie din X si linia corespunzatoare din Y

clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X, Y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [13]:
# ## TODO

# df = pd.read_csv('second_attack.csv')

# X_test = df

# ## apply transformation methods as above

# pred = clf.predict(X_test)
df2 = pd.read_csv('12_attack.csv')
X_test = df2
X_test.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,"2010-06-12 14:12:00,237404",67.212.184.66,192.168.1.101,TCP,60,80 > 2301 [ACK] Seq=1 Ack=1 Win=5840 Len=0
1,2,"2010-06-12 14:12:00,237612",192.168.1.101,67.212.184.66,TCP,60,[TCP ZeroWindow] [TCP ACKed unseen segment] 23...
2,3,"2010-06-12 14:12:01,796591",192.168.2.113,69.48.248.51,HTTP,113,GET /r/publishers HTTP/1.1
3,4,"2010-06-12 14:12:01,905063",69.48.248.51,192.168.2.113,TCP,1434,80 > 2401 [ACK] Seq=1 Ack=60 Win=4356 Len=13...
4,5,"2010-06-12 14:12:01,905081",69.48.248.51,192.168.2.113,TCP,203,"80 > 2401 [PSH, ACK] Seq=1381 Ack=60 Win=435..."


In [14]:
df2.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()
d2=df2[(df2.Destination=='69.192.24.88') | (df2.Source=='69.192.24.88')] 

In [22]:
X_test=df2
X_test['Protocol'] = X_test['Protocol'].astype('category')
X_test['Destination'] = X_test['Destination'].astype('category')
X_test['Source'] = X_test['Source'].astype('category')
X_test['Info'] = X_test['Info'].astype('category')  # ToDo split info in relevant columns
X_test = X_test.drop(['Time'], axis=1)
X_test.head()
pred = clf.predict(X_test)

ValueError: could not convert string to float: '67.212.184.66'

In [19]:
trace3=go.Scatter(x=d2['Time'],y=d2['Length'],mode='markers')
l=d2.query('(Length>300)&(Length<600)')
trace4=go.Scatter(x=l['Time'],y=l['Length'], mode='lines')
data2=[trace3, trace4]
py.iplot(data2)

In [ ]:
## TODO
## Compute 'd' the same way you computed it in the first case
## Plot d + pred
## Compare overalp between d and pred